In [4]:
import torch.nn as nn
import torch
import torchvision
from torchvision import transforms
import os

In [2]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:
    dev = "cpu"  
    
device = torch.device(dev)  
a = torch.zeros(4,3)    
a = a.to(device)

In [3]:
a

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')

In [5]:
os.listdir('D://ML DL AI DSBDA//Mushroom Classification using Deep Learning//archive//Mushrooms//')

['Agaricus',
 'Amanita',
 'Boletus',
 'Cortinarius',
 'Entoloma',
 'Hygrocybe',
 'Lactarius',
 'Russula',
 'Suillus']

In [6]:
training_transforms = transforms.Compose([transforms.ToTensor()])

In [7]:
train_dataset_path = 'D://ML DL AI DSBDA//Mushroom Classification using Deep Learning//archive//Mushrooms//'

In [8]:
train_dataset = torchvision.datasets.ImageFolder(root = train_dataset_path, transform = training_transforms)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 32, shuffle = False)

In [21]:
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for images, _ in loader:
        image_count_in_a_batch = images.size(0)
        images = images.view(image_count_in_a_batch, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += image_count_in_a_batch
        
    mean = mean/total_images_count
    std = std/total_images_count
    
    return mean, std

In [22]:
mean, std = get_mean_std(train_loader)
mean, std

(tensor([0.3913, 0.3696, 0.2827]), tensor([0.2292, 0.2094, 0.2030]))

In [23]:
# Set the desired split ratio (e.g., 80% for training, 20% for testing)
train_ratio = 0.8

# Set the random seed for reproducibility
random_seed = 42

# Define the data transformations
transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [27]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import random

In [28]:
# Load the entire dataset using ImageFolder
full_dataset = ImageFolder(root = train_dataset_path, transform = transform)

# Set the random seed
random.seed(random_seed)

# Determine the size of the training set based on the split ratio
train_size = int(train_ratio * len(full_dataset))
test_size = len(full_dataset) - train_size

In [29]:
# Create a list of indices for the entire dataset
indices = list(range(len(full_dataset)))

# Randomly shuffle the indices
random.shuffle(indices)

# Split the indices into training and testing indices
train_indices = indices[:train_size]
test_indices = indices[train_size:]

In [30]:
# Create the training and testing datasets using Subset
train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
test_dataset = torch.utils.data.Subset(full_dataset, test_indices)

# Create DataLoaders for the training and testing datasets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [32]:
def give_train_test_data(train_dataloader, test_dataloader):
    for images, labels in train_dataloader:
        # Move the batch of images and labels to the GPU if available
        train_images = images.to(device)
        train_labels = labels.to(device)

    # Iterate over the testing dataset
    for images, labels in test_dataloader:
        # Move the batch of images and labels to the GPU if available
        test_images = images.to(device)
        test_labels = labels.to(device)
        
    return train_images, train_labels, test_images, test_labels


In [33]:
train_images, train_labels, test_images, test_labels = give_train_test_data(train_dataloader, test_dataloader)

In [35]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.relu4 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu5 = nn.ReLU()
        self.fc3 = nn.Linear(64, 9)  # Output layer with 9 mushroom classes
        
    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.pool3(self.relu3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu4(self.fc1(x))
        x = self.relu5(self.fc2(x))
        x = self.fc3(x)
        return x


In [39]:
import torch.optim as optim

# Instantiate the ImageClassifier model
model = ImageClassifier()

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Iterate over the training dataset
    for images, labels in train_dataloader:
        # Move the batch of images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
    
    # Calculate the average loss for the epoch
    average_loss = running_loss / len(train_dataloader)
    
    # Print the average loss for the epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {average_loss:.4f}")
    
# Testing loop
model.eval()  # Set the model to evaluation mode

total_correct = 0
total_samples = 0

# Disable gradient computation
with torch.no_grad():
    # Iterate over the testing dataset
    for images, labels in test_dataloader:
        # Move the batch of images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Get the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the counts for correct predictions
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

# Calculate the accuracy
accuracy = total_correct / total_samples

# Print the accuracy
print(f"Testing Accuracy: {accuracy:.4f}")


Epoch 1/50, Loss: 1.9039
Epoch 2/50, Loss: 1.6791
Epoch 3/50, Loss: 1.4323
Epoch 4/50, Loss: 1.0634
Epoch 5/50, Loss: 0.5680
Epoch 6/50, Loss: 0.2322
Epoch 7/50, Loss: 0.1273
Epoch 8/50, Loss: 0.0522
Epoch 9/50, Loss: 0.0189
Epoch 10/50, Loss: 0.0131
Epoch 11/50, Loss: 0.0511
Epoch 12/50, Loss: 0.1002
Epoch 13/50, Loss: 0.0391
Epoch 14/50, Loss: 0.0299
Epoch 15/50, Loss: 0.0262
Epoch 16/50, Loss: 0.0343
Epoch 17/50, Loss: 0.0143
Epoch 18/50, Loss: 0.0386
Epoch 19/50, Loss: 0.0371
Epoch 20/50, Loss: 0.0617
Epoch 21/50, Loss: 0.0451
Epoch 22/50, Loss: 0.0123
Epoch 23/50, Loss: 0.0009
Epoch 24/50, Loss: 0.0002
Epoch 25/50, Loss: 0.0001
Epoch 26/50, Loss: 0.0001
Epoch 27/50, Loss: 0.0001
Epoch 28/50, Loss: 0.0001
Epoch 29/50, Loss: 0.0001
Epoch 30/50, Loss: 0.0000
Epoch 31/50, Loss: 0.0000
Epoch 32/50, Loss: 0.0000
Epoch 33/50, Loss: 0.0000
Epoch 34/50, Loss: 0.0000
Epoch 35/50, Loss: 0.0000
Epoch 36/50, Loss: 0.0000
Epoch 37/50, Loss: 0.0000
Epoch 38/50, Loss: 0.0000
Epoch 39/50, Loss: 0.